GeoSpark Test
=================

Ref:
- https://datasystemslab.github.io/GeoSpark/tutorial/geospark-sql-python/
- https://github.com/DataSystemsLab/GeoSpark/blob/master/python/GeoSparkSQL.ipynb
- https://github.com/DataSystemsLab/GeoSpark/blob/master/python/GeoSparkCore.ipynb

In [1]:
from pyspark_utils import get_or_create_geospark_session
spark = get_or_create_geospark_session()
spark

In [2]:
import geopandas as gpd
# import pandas as pd

prepare `japan_ver82` from [esri-japan](https://www.esrij.com/products/japan-shp/)

In [3]:
gpdf = gpd.read_file('japan_ver82/japan_ver82.shp')
gpdf[:5]

,JCODE,KEN,SICHO,GUN,SEIREI,SIKUCHOSON,CITY_ENG,P_NUM,H_NUM,geometry
0,01101,北海道,石狩振興局,None,札幌市,中央区,"Sapporo-shi, Chuo-ku",235449.0,141734.0,"POLYGON ((141.34233 43.06682, 141.35520 43.068..."
1,01102,北海道,石狩振興局,None,札幌市,北区,"Sapporo-shi, Kita-ku",286112.0,151891.0,"POLYGON ((141.40839 43.18395, 141.40427 43.182..."
2,01103,北海道,石狩振興局,None,札幌市,東区,"Sapporo-shi, Higashi-ku",261777.0,142078.0,"POLYGON ((141.44698 43.15513, 141.44860 43.153..."
3,01104,北海道,石狩振興局,None,札幌市,白石区,"Sapporo-shi, Shiroishi-ku",212671.0,122062.0,"POLYGON ((141.46569 43.10000, 141.46812 43.097..."
4,01105,北海道,石狩振興局,None,札幌市,豊平区,"Sapporo-shi, Toyohira-ku",222504.0,126579.0,"POLYGON ((141.38479 43.04660, 141.38558 43.047..."


In [4]:
%%time
gdf = spark.createDataFrame(gpdf)
gdf.show()
gdf.dtypes

+-----+------+--------------------+----+------+----------+--------------------+--------+--------+--------------------+
|JCODE|   KEN|               SICHO| GUN|SEIREI|SIKUCHOSON|            CITY_ENG|   P_NUM|   H_NUM|            geometry|
+-----+------+--------------------+----+------+----------+--------------------+--------+--------+--------------------+
|01101|北海道|          石狩振興局|null|札幌市|    中央区|Sapporo-shi, Chuo-ku|235449.0|141734.0|POLYGON ((141.342...|
|01102|北海道|          石狩振興局|null|札幌市|      北区|Sapporo-shi, Kita-ku|286112.0|151891.0|POLYGON ((141.408...|
|01103|北海道|          石狩振興局|null|札幌市|      東区|Sapporo-shi, Higa...|261777.0|142078.0|POLYGON ((141.446...|
|01104|北海道|          石狩振興局|null|札幌市|    白石区|Sapporo-shi, Shir...|212671.0|122062.0|POLYGON ((141.465...|
|01105|北海道|          石狩振興局|null|札幌市|    豊平区|Sapporo-shi, Toyo...|222504.0|126579.0|POLYGON ((141.384...|
|01106|北海道|          石狩振興局|null|札幌市|      南区|Sapporo-shi, Mina...|137488.0| 72502.0|POLYGON ((141.102...|
|01107|北海道

[('JCODE', 'string'),
 ('KEN', 'string'),
 ('SICHO', 'string'),
 ('GUN', 'string'),
 ('SEIREI', 'string'),
 ('SIKUCHOSON', 'string'),
 ('CITY_ENG', 'string'),
 ('P_NUM', 'double'),
 ('H_NUM', 'double'),
 ('geometry', 'udt')]

In [5]:
%%time
spark.sql("create database if not exists tmp")
spark.sql("drop table if exists tmp.japan_ver82")
gdf.write.saveAsTable("tmp.japan_ver82")

CPU times: user 5.77 ms, sys: 65 µs, total: 5.84 ms
Wall time: 15 s


In [6]:
%%time
# check
_table = "tmp.japan_ver82"
spark.table(_table).show(5)
display(spark.table(_table).limit(5).toPandas())
display(spark.table(_table).dtypes)
del _table

+-----+------+-----+------+------+----------+--------------+-------+-------+--------------------+
|JCODE|   KEN|SICHO|   GUN|SEIREI|SIKUCHOSON|      CITY_ENG|  P_NUM|  H_NUM|            geometry|
+-----+------+-----+------+------+----------+--------------+-------+-------+--------------------+
|32207|島根県| null|  null|  null|    江津市|     Gotsu-shi|23664.0|11513.0|MULTIPOLYGON (((1...|
|32209|島根県| null|  null|  null|    雲南市|     Unnan-shi|38479.0|13786.0|MULTIPOLYGON (((1...|
|32343|島根県| null|仁多郡|  null|  奥出雲町|  Okuizumo-cho|12694.0| 4782.0|POLYGON ((133.078...|
|32386|島根県| null|飯石郡|  null|    飯南町|     Iinan-cho| 4898.0| 2072.0|POLYGON ((132.678...|
|32441|島根県| null|邑智郡|  null|    川本町|Kawamoto-machi| 3317.0| 1672.0|POLYGON ((132.487...|
+-----+------+-----+------+------+----------+--------------+-------+-------+--------------------+
only showing top 5 rows



,JCODE,KEN,SICHO,GUN,SEIREI,SIKUCHOSON,CITY_ENG,P_NUM,H_NUM,geometry
0,32207,島根県,None,None,None,江津市,Gotsu-shi,23664.0,11513.0,"(POLYGON ((132.283836498 35.05078841400007, 13..."
1,32209,島根県,None,None,None,雲南市,Unnan-shi,38479.0,13786.0,(POLYGON ((132.9725294420001 35.22926166700006...
2,32343,島根県,None,仁多郡,None,奥出雲町,Okuizumo-cho,12694.0,4782.0,"POLYGON ((133.0786477820001 35.25038555900005,..."
3,32386,島根県,None,飯石郡,None,飯南町,Iinan-cho,4898.0,2072.0,"POLYGON ((132.678336939 35.17283917100008, 132..."
4,32441,島根県,None,邑智郡,None,川本町,Kawamoto-machi,3317.0,1672.0,"POLYGON ((132.4872108300001 35.05614277500007,..."


[('JCODE', 'string'),
 ('KEN', 'string'),
 ('SICHO', 'string'),
 ('GUN', 'string'),
 ('SEIREI', 'string'),
 ('SIKUCHOSON', 'string'),
 ('CITY_ENG', 'string'),
 ('P_NUM', 'double'),
 ('H_NUM', 'double'),
 ('geometry', 'udt')]

CPU times: user 83.1 ms, sys: 11.8 ms, total: 94.8 ms
Wall time: 1.98 s
